# Generate CT slice video
Lung and abdomen windows with optional segmentations

In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
import os
import warnings
warnings.filterwarnings('ignore', category=SyntaxWarning)

from pathlib import Path
from concurrent.futures import ThreadPoolExecutor, wait

import arrow
import matplotlib
import matplotlib.pyplot as plt
import SimpleITK as sitk
from tqdm import tqdm

from utils.data import check_multiple_cts
from utils.video import create_mp4
from utils.plot import (
    plot_slice_full,
    plot_and_save,
    get_slice_range,
)

# matplotlib.use('Agg')  # Prevent figures from being cached

### Note
This code assumes there is 1 volume and 1 segmentation per subject. If there are more volumes per subject, keep only the one that's segmented (mentioned in the spreadsheet in the dataset)

In [ ]:
# input
project_folder = Path("/home/liushifeng/Desktop/AutoPET dataset")
dataset_folder = project_folder / "dataset B_melanoma"
seg_folder = project_folder / "SEG_Melanoma_B_Rifki"

suv_filename = "SUV.nii.gz"
ct_filename = "CT.nii.gz"
ct_paths = list(dataset_folder.glob(f"*/*/{ct_filename}"))
check_multiple_cts(ct_paths)

# output
output_folder = Path("../outputs")
frames_folder = output_folder / "frames"
videos_folder = output_folder / "videos"

## Generate video
Done concurrently, set the appropriate workers based on CPU cores

In [ ]:
# for testing
%matplotlib inline
plot_slice_full(ct_array, seg_array, suv_array, 300);

In [ ]:
max_workers = 31
exists = True
matplotlib.use('Agg');

for i, ct_path in enumerate(ct_paths):
    if "PETCT_d65cf203be" in str(ct_path):  # process will start after this
        exists = False
        continue
    if exists:
        continue

    # path manipulation
    dataset_name = ct_path.parts[-4]
    patient = ct_path.parts[-3]
    study = ct_path.parts[-2]
    print(f"[{i + 1:03d}/{len(ct_paths)}] Processing: {patient}/{study}")

    scan_hash = ct_path.parent.parent.stem.split("_")[1]
    seg_path = list(seg_folder.glob(f"*{scan_hash}*"))[0]
    suv_path = ct_path.parent / suv_filename

    # load data
    ct_img = sitk.ReadImage(ct_path)
    suv_img = sitk.ReadImage(suv_path)
    seg_img = sitk.ReadImage(seg_path)
    suv_img = sitk.Resample(suv_img, ct_img, sitk.Transform(), sitk.sitkLinear, 0.0, suv_img.GetPixelID())
    seg_img = sitk.Resample(seg_img, ct_img, sitk.Transform(), sitk.sitkNearestNeighbor, 0.0, seg_img.GetPixelID())

    ct_array = sitk.GetArrayFromImage(ct_img)
    suv_array = sitk.GetArrayFromImage(suv_img)
    seg_array = sitk.GetArrayFromImage(seg_img)

    slice_range, n_slices = get_slice_range(seg_array)

    # create frames folder
    video_name = f"{patient}__{study}"
    ct_frames_folder = frames_folder / dataset_name / video_name
    ct_frames_folder.mkdir(exist_ok=True, parents=True)

    # create videos folder
    date_str = arrow.now().format("YYYY-MM-DD")
    dataset_video_path = videos_folder / dataset_name
    dataset_video_path.mkdir(exist_ok=True, parents=True)
    output_path = dataset_video_path / f"{video_name}_video_{date_str}.mp4"

    print(f"Generating {n_slices} frames")
    # # no concurrency
    # for i in tqdm(range(*slice_range)):
    #     args = ct_array, seg_array, i, slice_range, ct_frames_folder
    #     plot_and_save(args)
    with ThreadPoolExecutor(max_workers=max_workers) as e:
        futures = []
        for i in range(*slice_range):
            args = ct_array, seg_array, suv_array, i, ct_frames_folder
            futures.append(e.submit(plot_and_save, args))
        wait(futures)

    print(f"Creating video: {output_path.name} from")
    create_mp4(output_path, ct_frames_folder)
    print(f"Video saved as {output_path}")
    print("-" * 30)

In [ ]:
os.listdir("../outputs/")

In [ ]:
# visualize a single slice
i = 880
fig = plot_slice_full(ct_array, seg_array, i, slice_range)
# fig.savefig(f"test2.jpg", bbox_inches='tight', pad_inches=0, dpi=180);

## Window and visualize

In [ ]:
# Select a slice for visualization (e.g., middle slice)
slice_idx = 1360

# Create overlay
# fig, ax = plt.subplots()
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(12, 6))

ax1.imshow(lung_ct[slice_idx], cmap='gray')
# ax1.imshow(seg_data[slice_idx], alpha=0.3, cmap='Reds')
ax1.set_title('Lung Window with Segmentation Overlay')

ax2.imshow(abdomen_ct[slice_idx], cmap='gray')
# ax2.imshow(seg_data[slice_idx], alpha=0.3, cmap='Reds')
ax2.set_title('Abdomen Window with Segmentation Overlay')

plt.show()

---
# Misc Scripts
## Rename files

In [ ]:
folder = Path("/home/liushifeng/Documents/GitHub/ctseg/outputs/dataset C_lymphoma/")
folder_names = os.listdir(folder)

for n in tqdm(folder_names):
    new_name = n.replace("dataset C_lymphoma__", "").replace("dataset C_lymphoma-", "")
    src = folder / n
    dst = folder / new_name
    os.rename(src, dst)